파이썬에서 속성을 설정할 때 특별한 동작이 일어나야 하면 @property 데코레이터와 이에 대응하는 setter 속성을 사용하는 방법으로 바꿀 수 있다.

In [2]:
class Resistor(object):
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0
        
r1 = Resistor(50e3)
r1.ohms = 10e3


Resistor의 새 서브클래스를 정의하여 voltage 프로퍼티를 할당하면 current 값이 바뀌게 해보자

In [4]:
class VoltageResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0
    
    @property
    def voltage(self):
        return self._voltage
    
    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self.ohms

In [5]:
r2 = VoltageResistance(1e3)
print('Before: %5r amps' % r2.current)
r2.voltage = 10
print('After: %5r amps' % r2.current)

Before:     0 amps
After:  0.01 amps


프로퍼티에 setter 설정하면 클래스에 전달된 값들의 타입을 체크하고 값을 검증할 수도 있다.

In [6]:
# BoundedResistance.__init__() -> Resistor.__init__() -> self.ohms = -5 -> @ohms.setter 호출
class BoundedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if ohms <= 0:
            raise ValueError('%f ohms must be > 0' % ohms)
        self._ohms = ohms

In [7]:
r3 = BoundedResistance(1e3)
r3.ohms = 0

ValueError: 0.000000 ohms must be > 0

생성자에 올바르지 않은 값을 넘겨도 예외가 일어난다

In [10]:
r4 = BoundedResistance(0)

ValueError: 0.000000 ohms must be > 0

부모 클래스의 속성을 불변(immutable)으로 만드는 데도 @property를 사용할 수 있다.

In [15]:
class FixedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
    
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if hasattr(self, '_ohms'):
            raise AttributeError("Can't set attribute")
        self._ohms = ohms

In [17]:
r4 = FixedResistance(1e3)
r4.ohms = 2

AttributeError: Can't set attribute

주의할점!!! @property 메서드로 세터와 게터를 구현할 때 예상과 다르게 동작하지 않게 해야한다.


In [31]:
class MysterioutResistor(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        
    @property
    def ohms(self):
        self.voltage = self._ohms * self.current
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        self._ohms = ohms

In [32]:
r6 = MysterioutResistor(2)
r6.current = 0.01
print('Before: %5r' % r6.voltage)
r6.ohms
print('After: %5r' % r6.voltage)

Before:     0
After:  0.02


위와 같이 게터 프로퍼티 메서드에서 다른 속성을 설정하지 말자